In [3]:
import serial
import serial.tools.list_ports
import io
import time
import numpy as np
import pandas as pd
from scipy import special
from scipy.signal import savgol_filter, find_peaks_cwt
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
from IPython import display
%matplotlib inline

plt.rcParams["figure.dpi"] = 150

## Funciones de adquisición

In [24]:
def input_port():
    '''Obtiene la lista de puertos, la presenta en pantalla y da 
    a elegir un puerto, devolviendo el string para conectarse
    '''
    ports = list(serial.tools.list_ports.comports())
    for i,p in enumerate(ports):
        print("[{}]: Puerto {}".format(i + 1, p[1]))
    port = ""
    if len(ports) > 0:
        port = ports[int(input("Ingrese el puerto serie: ")) - 1][0]
    return port

def update(s):
    '''Actualiza los datos del perfilador, en la propiedad data'''
    A = []
    t = time.clock()
    for i in range(500):
        A.append(s.readline().decode(errors="replace"))
        print(type(A[-1]))
    A = "".join(A)
    data = np.genfromtxt(io.StringIO(A), delimiter="\t", invalid_raise=False, loose = True)
    return data
    #data.t = pd.to_numeric(data.t, errors='coerce')
    #data.V = pd.to_numeric(data.V, errors='coerce')
    #return data.values

## Funciones de procesamiento de datos

In [18]:
err_f = lambda x, a, b, c, d: a + b * special.erf(np.sqrt(2)*(x - c) / d)

def process_data(data):
    T = data[:,0]
    V = data[:,1]

    #Obtengo automáticamente la frecuencia, a partir de FFT
    f = get_freq(data)

    filt = sp.signal.medfilt(V,kernel_size=25)
    gauss = np.abs(sp.ndimage.gaussian_filter(filt, 50, order=1))
    gauss = gauss/gauss.max()

    fig = plt.figure(1)
    plt.plot(T,V,'ro')
    #Plot de debuggeo
    #plt.plot(T,gauss,'bo')

    peaks = sp.signal.find_peaks_cwt(gauss, np.array([30]))
    plt.plot(T[peaks],V[peaks],'bd')
    err_f = lambda x, a, b, c, d: a + b * sp.special.erf(np.sqrt(2)*(x - c) / d)



    w = 2 * np.pi * f
    R = 10
    σ = []
    DeltaT = 0.2 / f #Encontrar frecuencia automáticamente!
    for i in peaks:
        ind = np.abs(T - T[i]) < DeltaT
        x = T[ind]
        y = V[ind]
        p0 = [y.max()/2, -y.max()/2, (x.max() + x.min())/2, 0.5*(x.max() - x.min())]
        try:
            p, cov = sp.optimize.curve_fit(err_f, x, y, p0 = p0)
            t = np.linspace(x.min(), x.max() , 1000)
            plt.plot(t, err_f(t,*p),figure=fig, linewidth=2)
            #print(p[3]*R*w)
            σ.append(p[3]*R*w)
        except RuntimeError:
            print("No se pudo ajustar")
            pass

    print(np.mean(np.array(σ)), np.sqrt(np.var(np.array(σ))))
    print(np.sqrt(np.var(np.array(σ)))/np.mean(np.array(σ)))
    
    x = data[:,0].copy()
    x -= x.min()
    x *= 1e-6
    y = data[:,1].copy()
    y_filtered = savgol_filter(y, 75, 7)
    diffY = np.abs(np.diff(y_filtered))
    diffY  = diffY/diffY.max()
    peakind = find_peaks_cwt(diffY, np.arange(10,50))
    Delta = 100

    lstData = []
    for num, i in enumerate(peakind):
        if y[i] > 0.2 * y.max() and y[i] < 0.9 * y.max():
            low = i - Delta if i - Delta > 0 else 0
            high = i + Delta if i + Delta < y.shape[0] else y.shape[0]
            lstData.append(data[low:high])
    return lstData

def fit_data(data):
    x = data[:,0]
    y = data[:,1]
    #Parámetros iniciales, dependiente de los datos
    p0 = [y.max()/2, -y.max()/2, (x.max() + x.min())/2, 0.5*(x.max() - x.min())]
    p, cov = curve_fit(err_f, x, y, p0 = p0)
    #Si la matriz de covarianza tiene elementos infinitos, los parámetros iniciales estaban mal
    #Vuelvo a ajusta con otra magnitud
    if np.any(np.isinf(cov)):
        p0 = [y.max()/2, y.max()/2, (x.max() + x.min())/2, 0.5*(x.max() - x.min())]
        p, cov = curve_fit(err_f, x, y, p0 = p0)
    
    t = np.linspace(x.min(), x.max(), 1000)
    return t, err_f(t,*p), p, cov

## Adquisición y graficación de datos

In [25]:
try:
    port = input_port()
    ser = serial.Serial(port, "115200")
    t = time.clock()
    while True:
        data = update(ser)
        
        #np.savetxt("../mediciones/med_10_11/data_15_57.csv", data)
        #display.clear_output()

        #plt.close()
        plt.plot(data, 'ro')
        #plt.show()
        #print(data.shape)
except KeyboardInterrupt:
    ser.close()
finally:
    ser.close()
    raise
#data = np.loadtxt("../mediciones/med_13_10/data_13_10_15.csv") #np.load("../mediciones/med_20_10/data_19_46.npy")

[1]: Puerto USB Serial Device (COM5)
[2]: Puerto Arduino Due Programming Port (COM6)
Ingrese el puerto serie: 1
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class

TypeError: Can't convert 'bytes' object to str implicitly

In [22]:
len(data)

716

In [ ]:
#data = np.loadtxt("../mediciones/med_10_11/data_15_57.csv")
A = process_data(data)
for i,a in enumerate(A):
    plt.figure(i)
    plt.plot(a[:,0], a[:,1], 'ro')
    t, F, p, cov = fit_data(a)
    print(p[3] * 10 * 2 * np.pi * 1e-6 * 10 * np.sqrt(2))
    plt.plot(t, F)

## Calibración

### He-Ne

In [ ]:
perfil = np.loadtxt("../mediciones/med_10_11/perfil_f_colimador.csv")
plt.plot(perfil[:,0], perfil[:,1], "ro")
p, cov = curve_fit(err_f, perfil[:,0], perfil[:,1])
t = np.linspace(perfil[:,0].min(), perfil[:,0].max(), 1000)
plt.plot(t, err_f(t, *p))
p[3] / 100 * 2